In [1]:
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
from scipy import stats, integrate
from scipy.optimize import minimize_scalar

%precision 3

'%.3f'

# 代表的な連続型確率分布

In [2]:
# 確率分布の性質を確認しやすくするための関数の定義
linestyles = ['-', '--', ':']


# 期待値を求める
def E(X, g=lambda x: x):
    x_range, f = X

    def integrand(x):
        return g(x) * f(x)

    return integrate.quad(integrand, -np.inf, np.inf)[0]


# 分散を求める
def V(X, g=lambda x: x):
    x_range, f = X
    mean = E(X, g)

    def integrand(x):
        return (g(x) - mean) ** 2 * f(x)

    return integrate.quad(integrand, -np.inf, np.inf[0])


# 確率変数を引数に、その確率変数が確率の性質を満たしているか確認し、期待値と分散を計算して返す
def check_prob(X):
    x_range, f = X
    f_min = minimize_scalar(f).fun
    assert f_min >= 0, '密度関数が負の値をとります'
    prob_sum = np.round(integrate.quad(f, -np.inf, np.inf)[0], 6)
    assert prob_sum == 1, f'確率の和が {prob_sum} になりました'
    print(f'期待値は {E(X):.3f}')
    print(f'分散は{V(X):.3f}')


# 確率変数と区間を引数に、その確率変数の密度関数と分布関数を図示する
def plot_prob(X, x_min, x_max):
    x_range, f = X

    def F(x):
        return integrate.quad(f, -np.inf, x)[0]

    xs = np.linspace(x_min, x_max, 100)
    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(111)
    ax = plot(xs, [f(x) for x in xs], label='f(x)', color='gray')
    ax.plot(xs, [F(x) for x in xs], label='F(x)', ls='--', color='gray')

    ax.legend()
    plt.show()